## And the walls kept tumbling down in the city that we love

In [1]:
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [2]:
rocks = [[eval(rock) for rock in line.split(' -> ')] for line in Path('rocks.txt').read_text().split('\n')]

minx, miny = [min([coord[i] for line in rocks for coord in line]) for i in range(2)]
maxx, maxy = [max([coord[i] for line in rocks for coord in line]) for i in range(2)]

print(f'x : {minx} -> {maxx}')
print(f'y : {miny} -> {maxy}')

x : 488 -> 552
y : 13 -> 157


In [3]:
hbuffer = 5
rock_locations = np.zeros((maxy+1, maxx-minx+1 + 2*hbuffer), dtype = int)
rock_locations[0, 500 - minx + hbuffer] = 4

for line in rocks:
    for i in range(1, len(line)):
        x1, y1 = line[i-1]
        x2, y2 = line[i]
        
        x1 -= minx
        x2 -= minx
        
        if x1 == x2:
            rock_locations[min(y1, y2):max(y1, y2) + 1, x1 + hbuffer] = '1'
        elif y1 == y2:
            rock_locations[y1, hbuffer + min(x1, x2):hbuffer + max(x1, x2) + 1] = '1'



In [4]:
plt.figure(figsize = (10, 10))
plt.imshow(rock_locations, interpolation = 'none')

In [5]:
sand_locations = np.copy(rock_locations)
sand_start = (0, 500 - minx + hbuffer)

oblivion = False

while not oblivion:
    rest = False
    current_loc = sand_start
    while not rest:
        y, x = current_loc
        
        below = sand_locations[y+1:, x]
        
        first_block = np.where(below > 0)[0]
        
        
        if len(first_block) == 0: # Nothing but air below
            oblivion = True
            rest = True
        else:
            first_block = first_block[0] + y + 1 # Find coordinate of first block
            
            if first_block == y + 1: # If first block is right below
                if x > 0 and sand_locations[y+1, x-1] == 0: # Attempt down-left
                    current_loc = (y+1, x-1)
                elif x < rock_locations.shape[1] - 1 and sand_locations[y+1, x+1] == 0: # Attempt down-right
                    current_loc = (y+1, x+1)
                else: # Rest
                    sand_locations[y, x] = 2
                    rest = True
            
            else: # Drop to right above first block
                current_loc = (first_block-1, x)

print(np.sum(sand_locations == 2))

719


In [6]:
plt.figure(figsize = (10, 10))
plt.imshow(sand_locations)

## Part 2: But if you close your eyes...

In [7]:
hbuffer = rock_locations.shape[0] + 2
rock_locations = np.zeros((maxy+3, maxx-minx+1 + 2*hbuffer), dtype = int)
rock_locations[0, 500 - minx + hbuffer] = 4
rock_locations[-1, :] = 1

for line in rocks:
    for i in range(1, len(line)):
        x1, y1 = line[i-1]
        x2, y2 = line[i]
        
        x1 -= minx
        x2 -= minx
        
        if x1 == x2:
            rock_locations[min(y1, y2):max(y1, y2) + 1, x1 + hbuffer] = '1'
        elif y1 == y2:
            rock_locations[y1, hbuffer + min(x1, x2):hbuffer + max(x1, x2) + 1] = '1'

In [8]:
plt.figure()
plt.imshow(rock_locations)

In [38]:
sand_locations = np.copy(rock_locations)
sand_start = (0, 500 - minx + hbuffer)

oblivion = False

frames = []

while not oblivion:
    rest = False
    current_loc = sand_start
    frames += [[current_loc, rest]]
    while not rest:
        y, x = current_loc

        below = sand_locations[y+1:, x]

        first_block = np.where(below > 0)[0]



        first_block = first_block[0] + y + 1 # Find coordinate of first block


        if first_block == y + 1: # If first block is right below
            if first_block == sand_locations.shape[0] - 1: #Rock bottom, rest
                sand_locations[y, x] = 2
                rest = True

            elif sand_locations[y+1, x-1] == 0: # Attempt down-left
                current_loc = (y+1, x-1)
            elif sand_locations[y+1, x+1] == 0: # Attempt down-right
                current_loc = (y+1, x+1)
            else: # Rest
                sand_locations[y, x] = 2
                rest = True

        else: # Drop to right above first block
            current_loc = (first_block-1, x)

        if sand_locations[sand_start] == 2:
            oblivion = True
            
        frames += [[current_loc, rest]]

print(np.sum(sand_locations == 2))

23390


In [10]:
plt.figure()
plt.imshow(sand_locations)

## Movie

In [11]:
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FFMpegWriter

In [57]:
plt.close('all')

animated_locations = np.copy(rock_locations)

fig, ax = plt.subplots(1, 1)

im = ax.imshow(animated_locations)

def update(frame):
    global animated_locations
    global im
    
    temp_locations = np.copy(animated_locations)
    
    coord, rest = frames[frame]
    
    if rest:
        animated_locations[coord] = 4
        im.set_data(animated_locations)
    else:
        temp_locations[coord] = 4
        im.set_data(temp_locations)
    
    return [im]

ani = FuncAnimation(fig, update, frames = len(frames), interval = 1, blit = True)


# Export: warning: super slow, limit the number of frames for your sanity
# plt.rcParams['animation.ffmpeg_path'] = 'C:/ffmpeg/bin/ffmpeg.exe'
# writer = FFMpegWriter(fps = 50)
# ani.save('test.mp4', writer = writer)